In [1]:
import numpy as np
import pandas as pd
import os
import re
from tqdm import tqdm

In [2]:
import dgl
import torch
import torch.nn as nn
from dgl.data import CoraGraphDataset


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl
import dgl.nn as dglnn
from create_graph_data import create_graph
from model_zoo.NMR_gcn import NMR_GCN
from dgl import AddSelfLoop
from train_evaluate import NMR_prediction
from sklearn.metrics import mean_squared_error
from biopandas.pdb import PandasPdb

In [4]:
pd.set_option('display.max_rows', 100+1)

In [5]:
experimental_data_dir = 'experimental_data/FullyAnnotatedPDB_V2/'

In [6]:
combined_files = os.listdir(experimental_data_dir)

In [7]:
ppdb_df = PandasPdb().read_pdb('experimental_data/FullyAnnotatedPDB_V2/DB12634.pdb')
HETATM_df = ppdb_df.df['HETATM']

In [8]:
HETATM_df.columns

Index(['record_name', 'atom_number', 'blank_1', 'atom_name', 'alt_loc',
       'residue_name', 'blank_2', 'chain_id', 'residue_number', 'insertion',
       'blank_3', 'x_coord', 'y_coord', 'z_coord', 'occupancy', 'b_factor',
       'blank_4', 'segment_id', 'element_symbol', 'charge', 'line_idx'],
      dtype='object')

In [9]:
# PandasPdb().read_pdb('experimental_data/FullyAnnotatedPDB_V2/a-D-Galp.pdb').df['HETATM']

##### 1, Extract linear glycans

In [10]:
linear_glycan_list = []
for i in range(len(combined_files)):
    
    temp_square_bracket_content = re.search(r"\[(.*?)\]", combined_files[i])
    bracket_list = re.findall("\((.*?)\)", combined_files[i])
    
    
    if 'DB' in combined_files[i]:
        continue
    
    elif temp_square_bracket_content is not None:
        continue
    
    elif bracket_list:
        for s in bracket_list:
            temp_m = re.search('[a-zA-Z]', s)
            # check branching, (O-1) etc. is fine 
            if temp_m and ('O-' not in s):
                print(combined_files[i], 'branching glycans:', s)
                
            else:
                linear_glycan_list.append(combined_files[i])
    else:
    

        linear_glycan_list.append(combined_files[i])
linear_glycan_list = np.unique(linear_glycan_list)

Galb3(Fuca4)GlcNAcb.csv branching glycans: Fuca4
Gala1-3(Fuca1-2)Galb.csv branching glycans: Fuca1-2
Galb3(Fuca4)GlcNAcb.pdb branching glycans: Fuca4
NeuAca2-3Galb1-3( NeuAca2-3Galb1-4GlcNAc b1-6)GalNAc.pdb branching glycans:  NeuAca2-3Galb1-4GlcNAc b1-6
NeuAca2-3Galb1-3( NeuAca2-3Galb1-4GlcNAc b1-6)GalNAc.csv branching glycans:  NeuAca2-3Galb1-4GlcNAc b1-6
Gala1-3(Fuca1-2)Galb.pdb branching glycans: Fuca1-2


##### 2 Find unmatched glycans

In [11]:
csv_list = []
pdb_list = []
for j in range(len(linear_glycan_list)):
    temp_f = linear_glycan_list[j]
    
    if '.csv' in temp_f:
        temp_f_replace_csv = temp_f.replace('.csv', '.pdb')
        temp_f_delete_csv = temp_f.replace('.csv', '')
        
        if temp_f_replace_csv in linear_glycan_list:
            csv_list.append(temp_f)
            pass
        else:
            print('')
            print('missing pdb files given .csv')
            print(temp_f)
        
    
    if '.pdb' in temp_f:
        
        temp_f_replace_pdb = temp_f.replace('.pdb', '.csv')
        temp_f_delete_pdb = temp_f.replace('.pdb', '')
    
        if temp_f_replace_pdb in linear_glycan_list:
            pdb_list.append(temp_f)
            pass
        else:
            print('')
            print('missing .csv files given .pdb')
            print(temp_f)


missing pdb files given .csv
Repeat-4)-a-D-Manp-(1-4)-a-D-GalpA-(1-3)-b-D-GlcpNAc-(1-2)-a-D-Galp-(1-3)-a-L-Rhap2Ac-(1-.csv

missing pdb files given .csv
Repeat-4)-b-D-GlcpA-(1-6)-a-D-Glcp-(1-4)-b-D-Galp-(1-4)-b-D-Glcp-(1-.csv


##### 3 Read in pdb file

In [12]:
def experiment_process_single_pdb(f):
    temp_shift = []
    for f_l in f:
        if 'HETATM' in f_l:
            temp_shift.append(f_l)

    new_shift = []
    for i in range(len(temp_shift)):
        temp_l = temp_shift[i].split(' ')
        temp_l_s = [i for i in temp_l if i != '']
        new_shift.append(temp_l_s[0:8])
    return new_shift

In [13]:
def extract_atom_type(f):
    temp_shift = []
    for f_l in f:
        if 'HETATM' in f_l:
            temp_shift.append(f_l)

    new_shift = []
    for i in range(len(temp_shift)):
        temp_l = temp_shift[i].split(' ')
        temp_l_s = [i for i in temp_l if i != '']
        new_shift.append(temp_l_s[-1].split('\n')[0])
    return new_shift

In [14]:
def handle_missing_residual_pdb(new_f):
    new_f_handle_missing_residual = []
    for i in range(len(new_f)):
        temp_l = new_f[i]
        try:
            fourth_element = int(temp_l[3])
            temp_l.insert(3, 'Missing monosaccharide')
            temp_l.pop()
            new_f_handle_missing_residual.append(temp_l)
        except:
            new_f_handle_missing_residual.append(temp_l)
    return new_f_handle_missing_residual

##### 3.1 Simplified version

In [15]:
# read in pdb
for i in range(len(pdb_list)):
    temp_f = pdb_list[i]
    with open(os.path.join(experimental_data_dir, temp_f)) as f:
        f = f.readlines()
        new_f = experiment_process_single_pdb(f)
        new_f = handle_missing_residual_pdb(new_f)
        new_df = pd.DataFrame(new_f)
        

        new_df.columns = ['HETATM', 'Atom_num', 'Atom_name', 'Residual_name', 'Residual_num', 'x', 'y', 'z']
        
        temp_f = temp_f.replace('.pdb', '.pdb.csv')
        new_df.to_csv(os.path.join('experimental_data/FullyAnnotatedPDB_pdb/', temp_f), index = False)

##### 3.2 extract atom

In [17]:
for i in range(len(pdb_list)):
    temp_f = pdb_list[i]
    with open(os.path.join(experimental_data_dir, temp_f)) as f:
        f = f.readlines()
        atoms = extract_atom_type(f)
        new_f = experiment_process_single_pdb(f)
        new_f = handle_missing_residual_pdb(new_f)
        new_df = pd.DataFrame(new_f)
        

        new_df.columns = ['HETATM', 'Atom_num', 'Atom_name', 'Residual_name', 'Residual_num', 'x', 'y', 'z']
        
        assert(len(new_df) == len(atoms))
        
        temp_f = temp_f.replace('.pdb', '.pdb.csv')
        
        new_df['atoms_simplify'] = atoms
        
        new_df.to_csv(os.path.join('experimental_data/FullyAnnotatedPDB_pdb_added_atom/', temp_f), index = False)

In [19]:
temp_f

'repeat-4)-b-D-ManpNAc-(1-4)-a-D-GlcpNAc-(1-.pdb.csv'

##### 4 Read in csv

In [15]:
columns_list = []
columns_length = []
for j in range(len(csv_list)):
    temp_f = csv_list[j]
    

    
    temp_df = pd.read_csv(os.path.join(experimental_data_dir, temp_f))
    idx = np.where(temp_df.loc[:, 'MHz'].values == 'Residue')[0].reshape(-1)
    
    if len(idx) == 2:
        idx = idx[-1]
    elif len(idx) == 1:
        idx = idx[0]
    columns_list.append(idx)
    
    idx_start = idx+1
    
    temp_df_reformulate = temp_df.loc[idx_start:, :]
    
    temp_df_reformulate.index= range(len(temp_df_reformulate))
    temp_df_reformulate.columns = ['Residual', 'Connection', 'Atom', 'Shift', '4', '5', '6', '7']
    
    temp_df_reformulate = temp_df_reformulate.loc[:, ['Residual', 'Connection', 'Atom', 'Shift']]
    
    temp_df_reformulate = temp_df_reformulate.dropna(axis = 0, how = 'all')
    temp_df_reformulate['Atom'] = [v.lstrip() for v in temp_df_reformulate['Atom'].values]
    
#     for v in temp_df_reformulate['Atom'].values:
#         if isinstance(v, float):
#             print(temp_f)
    
    l = len(temp_df_reformulate.columns)
    columns_length.append(l)
    if l == 9:
        print(temp_f)
    if 'Repeat-4)-a-L-IdopA-(1-4)-a-D-GlcpNAc-(1' in temp_f:
        w = temp_df_reformulate
        
        print(temp_f)
#     print(temp_df_reformulate.loc[:,'Residual'].values)
#     print('')
    temp_df_reformulate.to_csv(os.path.join('experimental_data/FullyAnnotatedPDB_label/', temp_f), index = False)

Repeat-4)-a-L-IdopA-(1-4)-a-D-GlcpNAc-(1-.csv


##### Assign residuals

In [16]:
# with open('experimental_data/record_preprocess/temp_exclude.txt') as f:
#     f = f.readlines()
# exclude_csv_list = [i.split('\n')[0] for i in f]

In [17]:
# normal preprocessing, all C appears before all H
# Contain C1 or H1, H11, H12 as initial
def assign_label_normal_glycan(temp_df, H_initial = ['H11', 'H12', 'H1'], C_initial = ['C1']):
    
    residual_list = temp_df['Residual'].values
    connection_list = temp_df['Connection'].values
    atom_list = temp_df['Atom'].values

    H_res = 0
    C_res = 0

    previous_residual = 'some residual'
    previous_connection = 'some connection'
    previous_atom = 'some atom'

    residual_list = []

    for i in range(len(temp_df)):
        current_residual = temp_df.loc[i, :].Residual
        current_connection = temp_df.loc[i, :].Connection
        current_atom = temp_df.loc[i, :].Atom

        if (current_atom in H_initial) and (previous_atom not in H_initial):
            H_res += 1

        if (C_res == 0) and (current_atom not in C_initial):
            residual_list.append(H_res)
        else:
            if current_atom in C_initial:
                C_res += 1
            residual_list.append(C_res)

        previous_residual = current_residual
        previous_connection = current_connection
        previous_atom = current_atom


    temp_df_out = temp_df.copy()
    temp_df_out['Residual Num'] = residual_list

    return temp_df_out

In [18]:
def assign_label_wierd_glycan_by_connection(temp_df):
    H_initial = ['H11', 'H12', 'H1', 'H31', 'H32', 'H3 a', 'H1 a']
    C_initial = ['C1']

    residual_list = temp_df['Residual'].values
    connection_list = temp_df['Connection'].values
    atom_list = temp_df['Atom'].values

    H_res = 0
    C_res = 0

    previous_residual = 'some residual'
    previous_connection = 'some connection'
    previous_atom = 'some atom'

    residual_list = []

    for i in range(len(temp_df)):
        current_residual = temp_df.loc[i, :].Residual
        current_connection = temp_df.loc[i, :].Connection
        current_atom = temp_df.loc[i, :].Atom

        if (current_atom in H_initial) and (previous_atom not in H_initial):
            H_res += 1

        if (C_res == 0) and (current_atom not in C_initial):
            residual_list.append(H_res)
        else:
            if (current_atom in C_initial):
                C_res += 1
            elif (C_res != 0) and (current_connection != previous_connection):
                C_res += 1
            residual_list.append(C_res)

        previous_residual = current_residual
        previous_connection = current_connection
        previous_atom = current_atom


    temp_df_out = temp_df.copy()
    temp_df_out['Residual Num'] = residual_list
    return temp_df_out

In [19]:
exclude_csv_list = ['a-D-Kdop-(2-8)-a-D-Kdop-(2-4)-a-D-Kdop-(2-6)-b-D-GlcpNAc-(1-1)-Allyl.csv',
'a-D-Kdop-(2-8)-a-D-Kdop-(2-4)-a-D-Kdop-(2-6)-b-D-GlcpN4PO4-(1-6)-a-D-GlcpN1PO4.csv',
'a-D-Kdop-(2-4)-a-D-Kdop-(2-1)-Allyl.csv',
'a-D-Kdop-(2-8)-a-D-Kdop-(2-4)-a-D-Kdop-(2-6)-b-D-GlcpN-(1-6)-a-D-GlcpN1PO4.csv',
'b-D-Kdop-(2-3)-b-D-Galp-(1-4)-a-D-Galp-(1-3)-b-D-Galp.csv',
'a-D-Kdop-(2-6)-b-D-GlcpNAc-(1-1)-Allyl.csv',
'a-D-Kdop-(2-4)-a-D-Kdop-(2-6)-b-D-GlcpNAc-(1-1)-Allyl.csv',
'a-D-Kdop-(2-8)-a-D-Kdop-(2-4)-a-D-Kdop-(2-6)-b-D-GlcpNAc-(1-6)-a-D-GlcpNAc-(1-1)-Allyl.csv',
'a-D-Kdop-(2-8)-a-D-Kdop-(2-4)-a-D-Kdop-(2-6)-b-D-GlcpNAc-(1-6)-b-D-GlcpNAc-(1-1)-Allyl.csv',
'a-D-Kdop-(2-4)-a-D-Kdop-(2-6)-b-D-GlcpNAc-(1-6)-b-D-GlcpNAc-(1-1)-Allyl.csv',
'a-D-Kdop-(2-8)-a-D-Kdop-(2-1)-Allyl.csv',
'a-D-GlcpNAc-(1-4)-b-D-Galp-(1-3)-D-GalNAc-ol.csv',
'a-D-Kdop-(2-6)-b-D-GlcpNAc-(1-6)-a-D-GlcpNAc-(1-1)-Allyl.csv',
'a-D-GlcpNAc-(1-4)-a-D-GlcpNAc-(1-4)-a-D-Kdop.csv',
'a-D-Kdop-(2-4)-a-D-Kdop-(2-6)-b-D-GlcpNAc-(1-6)-a-D-GlcpNAc-(1-1)-Allyl.csv',
'a-D-Manp-(1-2)-a-D-Manp-(1-3)-Ser.csv',
'a-D-Glcp-(1-2)-a-D-Glcp-(1-2)-b-D-Fruf-(1-2)-b-D-Fruf.csv',
'a-L-Fucp-(1-2)-b-D-Galp-(1-3)-D-GalNAc-ol.csv',
'repeat-4)-b-D-ManpNAc-(1-4)-a-D-GlcpNAc-(1-.csv',
'a-D-Kdop-(2-4)-a-D-Kdop-(2-1)-Allyl.csv',
'b-D-Fruf-(2-6)-b-D-Fruf.csv', 
'a-D-Kdop-(2-6)-b-D-GlcpNAc-(1-6)-b-D-GlcpNAc-(1-1)-Allyl.csv']

In [20]:
reformulated_csv_dir = 'experimental_data/FullyAnnotatedPDB_label/'
reformulated_csv_list = os.listdir(reformulated_csv_dir)

In [21]:
def count_residual_from_swecon(csv_name, orig_dir = 'experimental_data/FullyAnnotatedPDB_V2/'):
    pdb_name = csv_name.replace('csv', 'pdb')
    max_val = 0
    l_f = []
    l_t = []
    with open(os.path.join(orig_dir, pdb_name)) as f:
        f = f.readlines()
        for l in f:
            if 'SWECON' in l:
                l1 = l.split(' ')
                l1 = [i for i in l1 if i != '']
#                 print(l1, csv_name, len(l1))
                if len(l1) == 5:
                    l_f.append(int(l1[2]))
                    l_t.append(int(l1[3]))
                elif len(l1) == 4:
                    l_f.append(int(l1[1]))
                    l_t.append(int(l1[2]))
    max_val = max(max(l_f), max(l_t))
    return max_val

In [22]:
def check_carbon_hydrogen_same(df_out, print_anyway=False):
    df_atom_list = df_out['Atom'].values
    df_res_list = df_out['Residual Num'].values
    max_h = 1
    max_c = 1
    for i in range(len(df_atom_list)):
        if 'H' in df_atom_list[i]:
            if df_res_list[i] > max_h:
                max_h = df_res_list[i]
        if 'C' in df_atom_list[i]:
            if df_res_list[i] > max_c:
                max_c = df_res_list[i]
    if max_c != max_h:
        print(csv_f)
        print('carbon:', max_c, 'hydrogen:', max_h)
        print('')
    if print_anyway:
        print(csv_f)
        print('carbon:', max_c, 'hydrogen:', max_h)
        print('')
    
    return max_c, max_h

In [23]:
for csv_name in reformulated_csv_list:
    temp_df = pd.read_csv(os.path.join(reformulated_csv_dir, csv_name))
    
    if csv_name not in exclude_csv_list:
        df_out = assign_label_normal_glycan(temp_df)
    else:
        df_out = assign_label_normal_glycan(temp_df, 
                                            H_initial = ['H11', 'H12', 'H1', 'H31', 'H32', 'H3 a', 'H1 a'], 
                                            C_initial = ['C1'])

        if csv_name == 'a-D-Kdop-(2-4)-a-D-Kdop-(2-6)-b-D-GlcpNAc-(1-1)-Allyl.csv':
            df_out.loc[[3, 4], ['Residual Num']] = [1, 1]
        
        if csv_name == 'a-D-Kdop-(2-8)-a-D-Kdop-(2-4)-a-D-Kdop-(2-6)-b-D-GlcpNAc-(1-1)-Allyl.csv':
            df_out.loc[[3, 4], ['Residual Num']] = [1, 1]
        
        if csv_name == 'a-D-Kdop-(2-8)-a-D-Kdop-(2-4)-a-D-Kdop-(2-6)-b-D-GlcpN4PO4-(1-6)-a-D-GlcpN1PO4.csv':
            temp_df = pd.read_csv(os.path.join(reformulated_csv_dir, csv_name))
            df_out = assign_label_normal_glycan(temp_df, 
                                            H_initial = ['H11', 'H12', 'H1', 'H2', 'H31', 'H32', 'H3 a', 'H1 a'], 
                                            C_initial = ['C1'])
        
        if csv_name == 'a-D-Kdop-(2-4)-a-D-Kdop-(2-1)-Allyl.csv':
            df_out.loc[[3, 4], ['Residual Num']] = [1, 1]
            
            
        if csv_name == 'a-D-Kdop-(2-8)-a-D-Kdop-(2-4)-a-D-Kdop-(2-6)-b-D-GlcpN-(1-6)-a-D-GlcpN1PO4.csv':
            temp_df = pd.read_csv(os.path.join(reformulated_csv_dir, csv_name))
            df_out = assign_label_wierd_glycan_by_connection(temp_df)
            
        if csv_name == 'a-D-Kdop-(2-6)-b-D-GlcpNAc-(1-1)-Allyl.csv':
            df_out.loc[[3, 4], ['Residual Num']] = [1, 1]
        
        if csv_name == 'a-D-Kdop-(2-8)-a-D-Kdop-(2-1)-Allyl.csv':
            df_out.loc[[3, 4], ['Residual Num']] = [1, 1]

        if csv_name == 'a-D-GlcpNAc-(1-4)-b-D-Galp-(1-3)-D-GalNAc-ol.csv':
            temp_df = pd.read_csv(os.path.join(reformulated_csv_dir, csv_name))
            df_out = assign_label_normal_glycan(temp_df, 
                                H_initial = ['H11', 'H12', 'H1', 'H2', 'H31', 'H32', 'H3 a', 'H1 a'], 
                                C_initial = ['C1'])
        if csv_name == 'a-D-Manp-(1-2)-a-D-Manp-(1-3)-Ser.csv':
            temp_df = pd.read_csv(os.path.join(reformulated_csv_dir, csv_name))
            df_out = assign_label_normal_glycan(temp_df, 
                    H_initial = ['HA', 'H1'], 
                    C_initial = ['C1', 'CO'])
#         print(csv_name)
#         print(df_out)
#         print('')
#     max_c, max_h = check_carbon_hydrogen_same(df_out, print_anyway=False)
#     total_res = count_residual_from_swecon(csv_name)
    
#     if (total_res != max_c) and (total_res != max_h):
#         print(csv_name)
    
    df_out.to_csv(os.path.join('experimental_data/FullyAnnotatedPDB_label_assigned_residual_num/', csv_name), index = False)